# Theoretical Autonomous Satellite Collision Avoidance (TASCA)
**Scope:** lightweight, Python-based prototype for autonomous collision avoidance in low-Earth orbit using a supervised imitation approach (oracle → neural network) with 3D visualization.

**What this notebook contains**
1. Clohessy-Wiltshire relative-motion simulator (LVLH frame) with RK4 propagation.
2. A simple rule-based oracle that computes a single impulsive ∆v to raise the closest-approach distance.
3. Synthetic dataset generation.
4. A small feedforward neural network (numpy) trained to imitate the oracle.
5. 3D visualization of nominal vs NN-avoided vs oracle-avoided trajectories using `matplotlib`.
6. Save/load checkpoint utilities.


## Requirements

This notebook uses only widely available Python packages: `numpy` and `matplotlib`. Install them if needed:

```bash
pip install numpy matplotlib nbformat
```


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math, os, json
np.random.seed(42)

print('numpy version', np.__version__)


numpy version 2.0.2


## Clohessy-Wiltshire dynamics and trajectory propagation

We implement CW linearized relative-motion dynamics in the Local-Vertical Local-Horizontal (LVLH) frame and integrate them using RK4.
The state vector is `[rx, ry, rz, vx, vy, vz]` where distances are in kilometers and velocities in km/s.


In [2]:
def cw_dot(s, n):
    rx, ry, rz, vx, vy, vz = s
    ax = 3 * n**2 * rx + 2 * n * vy
    ay = -2 * n * vx
    az = -n**2 * rz
    return np.array([vx, vy, vz, ax, ay, az])

def cw_propagate(state, dt, n):
    # RK4 integrator for a single CW step
    s = state.copy()
    k1 = cw_dot(s, n)
    k2 = cw_dot(s + 0.5 * dt * k1, n)
    k3 = cw_dot(s + 0.5 * dt * k2, n)
    k4 = cw_dot(s + dt * k3, n)
    return s + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

def propagate_trajectory(initial_state, dt, steps, n, impulsive_dv_time=None, dv=np.zeros(3)):
    traj = np.zeros((steps+1, 6))
    traj[0] = initial_state.copy()
    for i in range(steps):
        if impulsive_dv_time is not None and i == impulsive_dv_time:
            traj[i, 3:6] += dv
        traj[i+1] = cw_propagate(traj[i], dt, n)
    return traj


### Utility functions


In [3]:
def compute_min_distance_and_time(traj_chaser, traj_debris):
    dists = np.linalg.norm(traj_chaser[:,0:3] - traj_debris[:,0:3], axis=1)
    idx = np.argmin(dists)
    return dists[idx], idx

def generate_scenario(n=0.0011, dt=10.0, max_rel=2.0):
    rx = np.random.uniform(-max_rel, max_rel)
    ry = np.random.uniform(-max_rel, max_rel)
    rz = np.random.uniform(-max_rel/2, max_rel/2)
    vx = np.random.uniform(-0.01, 0.01)
    vy = np.random.uniform(-0.01, 0.01)
    vz = np.random.uniform(-0.005, 0.005)
    chaser = np.array([rx, ry, rz, vx, vy, vz])
    dr = chaser + np.array([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), np.random.uniform(-0.2, 0.2),
                            np.random.uniform(-0.005, 0.005), np.random.uniform(-0.005, 0.005), np.random.uniform(-0.002, 0.002)])
    return chaser, dr


## Oracle impulsive avoidance policy

The oracle simulates both chaser and debris forward for a short lookahead and, if the closest-approach distance falls below a threshold, computes a single impulsive ∆v. The ∆v direction is chosen to be approximately perpendicular to the relative velocity to increase miss distance.


In [4]:
def oracle_impulsive_action(chaser_state, debris_state, n, dt=1.0, lookahead_steps=200, threshold=0.75):
    traj_c = propagate_trajectory(chaser_state, dt, lookahead_steps, n)
    traj_d = propagate_trajectory(debris_state, dt, lookahead_steps, n)
    min_dist, idx = compute_min_distance_and_time(traj_c, traj_d)
    if min_dist >= threshold:
        return np.zeros(3), min_dist, idx
    rel_vel = traj_c[idx, 3:6] - traj_d[idx, 3:6]
    rel_pos = traj_c[idx, 0:3] - traj_d[idx, 0:3]
    perp = np.cross(rel_vel, rel_pos)
    if np.linalg.norm(perp) < 1e-6:
        perp = np.array([-rel_vel[1], rel_vel[0], 0.0])
    dv_dir = np.cross(rel_vel, perp)
    if np.linalg.norm(dv_dir) == 0:
        dv_dir = perp
    dv_dir = dv_dir / np.linalg.norm(dv_dir)
    needed = threshold - min_dist
    rel_speed = max(np.linalg.norm(rel_vel), 1e-3)
    k = 0.6
    dv_mag = k * needed * rel_speed
    dv = dv_mag * dv_dir
    return dv, min_dist, idx


## Synthetic dataset generation

We collect features `[chaser_pos, chaser_vel, debris_pos, debris_vel]` (12 numbers) and labels `∆v` (3 numbers) from the oracle.


In [5]:
def build_dataset(N=400, n=0.0011, dt=10.0):
    X = []
    Y = []
    stats = []
    for i in range(N):
        ch, de = generate_scenario(n=n, dt=dt)
        dv, min_dist, t_idx = oracle_impulsive_action(ch, de, n, dt=1.0, lookahead_steps=200, threshold=0.75)
        feat = np.concatenate([ch[0:3], ch[3:6], de[0:3], de[3:6]])
        X.append(feat); Y.append(dv); stats.append(min_dist)
    return np.array(X), np.array(Y), np.array(stats)

# Tune N higher for better performance
N = 400
X, Y, stats = build_dataset(N=N)
print('X shape:', X.shape, 'Y shape:', Y.shape)
print('mean min-dist in dataset: {:.3f} ± {:.3f}'.format(stats.mean(), stats.std()))


X shape: (400, 12) Y shape: (400, 3)
mean min-dist in dataset: 0.347 ± 0.147


## Feedforward neural network

A single-hidden-layer network with tanh activation.


In [6]:
class SimpleNN:
    def __init__(self, input_dim, hidden_dim, output_dim, lr=1e-3):
        self.w1 = np.random.randn(input_dim, hidden_dim) * np.sqrt(2/input_dim)
        self.b1 = np.zeros(hidden_dim)
        self.w2 = np.random.randn(hidden_dim, output_dim) * np.sqrt(2/hidden_dim)
        self.b2 = np.zeros(output_dim)
        self.lr = lr
    def forward(self, x):
        z1 = x.dot(self.w1) + self.b1
        a1 = np.tanh(z1)
        out = a1.dot(self.w2) + self.b2
        cache = (x, z1, a1)
        return out, cache
    def predict(self, x):
        out, _ = self.forward(x)
        return out
    def train_step(self, x, y):
        out, cache = self.forward(x)
        x_in, z1, a1 = cache
        err = out - y
        loss = np.mean(np.sum(err**2, axis=1))
        grad_out = (2.0 / x.shape[0]) * err
        grad_w2 = a1.T.dot(grad_out); grad_b2 = grad_out.sum(axis=0)
        grad_a1 = grad_out.dot(self.w2.T)
        grad_z1 = grad_a1 * (1 - np.tanh(z1)**2)
        grad_w1 = x_in.T.dot(grad_z1); grad_b1 = grad_z1.sum(axis=0)
        self.w1 -= self.lr * grad_w1; self.b1 -= self.lr * grad_b1
        self.w2 -= self.lr * grad_w2; self.b2 -= self.lr * grad_b2
        return loss


## Training the network

We normalize inputs and outputs, train, and observe loss.


In [ ]:
X_mean = X.mean(axis=0); X_std = X.std(axis=0) + 1e-8
Y_mean = Y.mean(axis=0); Y_std = Y.std(axis=0) + 1e-8
Xn = (X - X_mean) / X_std
Yn = (Y - Y_mean) / Y_std

model = SimpleNN(input_dim=Xn.shape[1], hidden_dim=64, output_dim=3, lr=8e-4)

# Tune these higher for stronger models
epochs = 60
batch = 64

for e in range(epochs):
    perm = np.random.permutation(Xn.shape[0])
    Xn = Xn[perm]; Yn = Yn[perm]
    losses = []
    for i in range(0, Xn.shape[0], batch):
        xb = Xn[i:i+batch]; yb = Yn[i:i+batch]
        loss = model.train_step(xb, yb); losses.append(loss)
    if (e+1) % 10 == 0 or e==0 or e==epochs-1:
        print(f'Epoch {e+1}/{epochs} loss={np.mean(losses):.6f}')


## 3D visualization: nominal vs NN vs oracle

The plots show debris trajectory and three chaser trajectories:
- nominal (no avoidance)
- NN avoidance (impulse at t=0 predicted by the network)
- oracle avoidance (the rule-based oracle's impulse)


In [ ]:
def run_visualization_case(chaser, debris, model, X_mean, X_std, Y_mean, Y_std, n=0.0011, dt=10.0):
    steps = 200
    traj_c_nom = propagate_trajectory(chaser, dt, steps, n)
    traj_d = propagate_trajectory(debris, dt, steps, n)
    min_nom, t_nom = compute_min_distance_and_time(traj_c_nom, traj_d)

    feat = np.concatenate([chaser[0:3], chaser[3:6], debris[0:3], debris[3:6]])
    x_in = (feat - X_mean) / X_std
    dv_pred_norm = model.predict(x_in.reshape(1,-1))[0]
    dv_pred = dv_pred_norm * Y_std + Y_mean

    traj_c_after = propagate_trajectory(chaser, dt, steps, n, impulsive_dv_time=0, dv=dv_pred)
    min_after, t_after = compute_min_distance_and_time(traj_c_after, traj_d)
    dv_oracle, min_oracle, t_oracle = oracle_impulsive_action(chaser, debris, n, dt=1.0, lookahead_steps=200, threshold=0.75)
    traj_oracle = propagate_trajectory(chaser, dt, steps, n, impulsive_dv_time=0, dv=dv_oracle)

    print(f'Nominal min-dist: {min_nom:.3f} at t={t_nom}, after NN: {min_after:.3f} at t={t_after}, oracle min-dist: {min_oracle:.3f}')

    fig = plt.figure(figsize=(12,5))
    ax = fig.add_subplot(121, projection='3d')
    ax.plot(traj_d[:,0], traj_d[:,1], traj_d[:,2], label='Debris')
    ax.plot(traj_c_nom[:,0], traj_c_nom[:,1], traj_c_nom[:,2], label='Chaser (nominal)')
    ax.scatter(traj_c_nom[t_nom,0], traj_c_nom[t_nom,1], traj_c_nom[t_nom,2], c='r', label='Closest nom')
    ax.set_title('Nominal (no avoidance)')
    ax.legend(); ax.set_xlabel('x (km)'); ax.set_ylabel('y (km)'); ax.set_zlabel('z (km)')
    ax.view_init(elev=20, azim=120)

    ax2 = fig.add_subplot(122, projection='3d')
    ax2.plot(traj_d[:,0], traj_d[:,1], traj_d[:,2], label='Debris')
    ax2.plot(traj_c_after[:,0], traj_c_after[:,1], traj_c_after[:,2], label='Chaser (NN avoidance)')
    ax2.plot(traj_oracle[:,0], traj_oracle[:,1], traj_oracle[:,2], label='Chaser (Oracle)')
    ax2.scatter(traj_c_after[t_after,0], traj_c_after[t_after,1], traj_c_after[t_after,2], c='g', label='Closest after')
    ax2.set_title('After avoidance (NN vs Oracle)')
    ax2.legend(); ax2.set_xlabel('x (km)'); ax2.set_ylabel('y (km)'); ax2.set_zlabel('z (km)')
    ax2.view_init(elev=20, azim=120)
    plt.tight_layout(); plt.show()

for i in range(3):
    ch, de = generate_scenario()
    print('\nCase', i+1)
    run_visualization_case(ch, de, model, X_mean, X_std, Y_mean, Y_std)


## 3D visualization: animation

This cell creates and saves a short animation that shows, side-by-side, how a debris object and the chaser move over time:

- left panel: debris (blue) + chaser nominal trajectory (orange)  
- right panel: debris (blue) + chaser with the NN-predicted impulse (orange) and oracle impulse (green)

**Output files**
- `avoidance_animation.mp4` - main video output (H.264 via `ffmpeg` if available)  
- `avoidance_animation.gif` - fallback GIF (may be large and lower quality)

**Notes**
- `ffmpeg` is recommended for good MP4 output. If it is not available, the code will try to save a GIF using Pillow.  
- The cell assumes the notebook already defines functions / variables from earlier cells:
  `generate_scenario`, `propagate_trajectory`, `oracle_impulsive_action`,
  and the trained `model` plus normalization arrays `X_mean, X_std, Y_mean, Y_std`.
- If `model` is not present, the code will fall back to using the oracle impulse for the "NN" trajectory so the animation still runs.
- Adjust `steps`, `dt`, and `interval` inside the code cell for longer/shorter animations or different temporal resolution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from IPython.display import HTML, display
import os

n = 0.0011         # mean motion used in earlier cells
dt = 10.0          # seconds per simulation step for visualization
steps = 200        # number of simulation steps (frame count)
interval = 50      # ms between frames (for display)
out_mp4 = 'avoidance_animation.mp4'
out_gif = 'avoidance_animation.gif'

chaser, debris = generate_scenario()

traj_d = propagate_trajectory(debris, dt, steps, n)

traj_nom = propagate_trajectory(chaser, dt, steps, n)

try:
    feat = np.concatenate([chaser[0:3], chaser[3:6], debris[0:3], debris[3:6]])
    x_in = (feat - X_mean) / X_std
    dv_pred_norm = model.predict(x_in.reshape(1,-1))[0]
    dv_pred = dv_pred_norm * Y_std + Y_mean
except Exception as e:
    print("Model unavailable or prediction failed; falling back to oracle for NN trace. Error:", e)
    dv_pred, _, _ = oracle_impulsive_action(chaser, debris, n, dt=1.0, lookahead_steps=200, threshold=0.75)

traj_nn = propagate_trajectory(chaser, dt, steps, n, impulsive_dv_time=0, dv=dv_pred)

dv_oracle, _, _ = oracle_impulsive_action(chaser, debris, n, dt=1.0, lookahead_steps=200, threshold=0.75)
traj_oracle = propagate_trajectory(chaser, dt, steps, n, impulsive_dv_time=0, dv=dv_oracle)

fig = plt.figure(figsize=(12,6))
ax_left  = fig.add_subplot(121, projection='3d')
ax_right = fig.add_subplot(122, projection='3d')

all_points = np.vstack([traj_d[:,0:3], traj_nom[:,0:3], traj_nn[:,0:3], traj_oracle[:,0:3]])
xmin, ymin, zmin = all_points.min(axis=0) - 0.5
xmax, ymax, zmax = all_points.max(axis=0) + 0.5

for ax in (ax_left, ax_right):
    ax.set_xlim(xmin, xmax); ax.set_ylim(ymin, ymax); ax.set_zlim(zmin, zmax)
    ax.set_xlabel('x (km)'); ax.set_ylabel('y (km)'); ax.set_zlabel('z (km)')

ax_left.set_title('Nominal (no avoidance)')
ax_right.set_title('After avoidance (NN vs Oracle)')

line_d_left, = ax_left.plot([], [], [], lw=2, label='Debris')
line_c_nom_left, = ax_left.plot([], [], [], lw=2, label='Chaser (nominal)', color='orange')
pt_closest_left, = ax_left.plot([], [], [], marker='x', color='red', linestyle='None', markersize=8)

line_d_right, = ax_right.plot([], [], [], lw=2, label='Debris')
line_c_nn_right, = ax_right.plot([], [], [], lw=2, label='Chaser (NN)', color='orange')
line_c_oracle_right, = ax_right.plot([], [], [], lw=2, label='Chaser (Oracle)', color='green')
pt_closest_right, = ax_right.plot([], [], [], marker='x', color='lime', linestyle='None', markersize=8)

ax_left.legend()
ax_right.legend()

def min_dist_index(traj1, traj2):
    dists = np.linalg.norm(traj1[:,0:3] - traj2[:,0:3], axis=1)
    idx = np.argmin(dists)
    return idx, dists[idx]

idx_nom, _ = min_dist_index(traj_nom, traj_d)
idx_after_nn, _ = min_dist_index(traj_nn, traj_d)
idx_oracle, _ = min_dist_index(traj_oracle, traj_d)

def update(frame):
    line_d_left.set_data(traj_d[:frame+1,0], traj_d[:frame+1,1])
    line_d_left.set_3d_properties(traj_d[:frame+1,2])
    line_c_nom_left.set_data(traj_nom[:frame+1,0], traj_nom[:frame+1,1])
    line_c_nom_left.set_3d_properties(traj_nom[:frame+1,2])
    if frame >= idx_nom:
        pt_closest_left.set_data([traj_nom[idx_nom,0]], [traj_nom[idx_nom,1]])
        pt_closest_left.set_3d_properties([traj_nom[idx_nom,2]])
    else:
        pt_closest_left.set_data([], []); pt_closest_left.set_3d_properties([])

    line_d_right.set_data(traj_d[:frame+1,0], traj_d[:frame+1,1])
    line_d_right.set_3d_properties(traj_d[:frame+1,2])
    line_c_nn_right.set_data(traj_nn[:frame+1,0], traj_nn[:frame+1,1])
    line_c_nn_right.set_3d_properties(traj_nn[:frame+1,2])
    line_c_oracle_right.set_data(traj_oracle[:frame+1,0], traj_oracle[:frame+1,1])
    line_c_oracle_right.set_3d_properties(traj_oracle[:frame+1,2])

    if frame >= idx_after_nn:
        pt_closest_right.set_data([traj_nn[idx_after_nn,0]], [traj_nn[idx_after_nn,1]])
        pt_closest_right.set_3d_properties([traj_nn[idx_after_nn,2]])
    else:
        pt_closest_right.set_data([], []); pt_closest_right.set_3d_properties([])

    return (line_d_left, line_c_nom_left, pt_closest_left,
            line_d_right, line_c_nn_right, line_c_oracle_right, pt_closest_right)

anim = animation.FuncAnimation(fig, update, frames=steps, interval=interval, blit=False)

saved = False
try:
    print("Saving MP4 (ffmpeg)...")
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=1000/interval, metadata=dict(artist='Notebook'), bitrate=8000)
    anim.save(out_mp4, writer=writer)
    print("Saved", out_mp4)
    saved = True
except Exception as e:
    print("ffmpeg save failed or not available:", e)

if not saved:
    try:
        from matplotlib.animation import PillowWriter
        print("Saving GIF (Pillow)...")
        writer = PillowWriter(fps=1000/interval)
        anim.save(out_gif, writer=writer)
        print("Saved", out_gif)
        saved = True
    except Exception as e:
        print("GIF save failed:", e)

plt.close(fig)

if os.path.exists(out_mp4):
    display(HTML(f"<h4>Animation (MP4)</h4><video controls src='{out_mp4}' width=800></video>"))
elif os.path.exists(out_gif):
    display(HTML(f"<h4>Animation (GIF)</h4><img src='{out_gif}' width=800>"))
else:
    print("No output file was saved. If running locally, install ffmpeg (recommended) or ensure Pillow is installed.")


## Save model checkpoint

This saves the learned weights and normalization statistics to a `.npz` file for later reuse.


In [ ]:
out_path = 'checkpoint.npz'
np.savez(out_path,
         w1=model.w1, b1=model.b1, w2=model.w2, b2=model.b2,
         X_mean=X_mean, X_std=X_std, Y_mean=Y_mean, Y_std=Y_std)
print('Saved checkpoint to', out_path)


## Load checkpoint and run single inference

This cell demonstrates how to load a saved checkpoint and run inference on one scenario.


In [ ]:
data = np.load('checkpoint.npz', allow_pickle=True)
w1 = data['w1']; b1 = data['b1']; w2 = data['w2']; b2 = data['b2']
X_mean = data['X_mean']; X_std = data['X_std']; Y_mean = data['Y_mean']; Y_std = data['Y_std']

class NNFromCheckpoint:
    def __init__(self, w1,b1,w2,b2):
        self.w1=w1; self.b1=b1; self.w2=w2; self.b2=b2
    def predict(self,x):
        z1 = x.dot(self.w1) + self.b1
        a1 = np.tanh(z1)
        return a1.dot(self.w2) + self.b2

nn = NNFromCheckpoint(w1,b1,w2,b2)

chaser, debris = generate_scenario()
feat = np.concatenate([chaser[0:3], chaser[3:6], debris[0:3], debris[3:6]])
x_in = (feat - X_mean) / X_std
dv_pred_norm = nn.predict(x_in.reshape(1,-1))[0]
dv_pred = dv_pred_norm * Y_std + Y_mean
print('Predicted ∆v (km/s):', dv_pred)


## Ingesting Two-Line Elements (TLEs) and converting to LVLH relative state

This cell obtains TLEs for two objects (a chaser and a target/debris) and converts them into a relative state vector in the LVLH frame:

- Step 1: fetch TLEs from CelesTrak by NORAD catalog ID or by satellite name. If fetching fails, the cell shows how to paste TLE lines manually.  
- Step 2: use `sgp4` to compute ECI (TEME) position and velocity at the current UTC time.  
- Step 3: convert ECI (TEME) outputs into an Earth-centered inertial (ECI) vector and then compute the LVLH basis from the chaser's position and velocity. Project the relative position and relative velocity into that basis to obtain the LVLH-relative state vector `[rx, ry, rz, vx, vy, vz]`.  
- Output: LVLH relative state arrays for chaser and target, which can be supplied to the notebook's simulator and ML pipeline.

Security note: public endpoints are used. If your environment blocks outgoing HTTP requests, manual TLE paste is supported.


In [ ]:
!pip install sgp4 requests
import requests
from sgp4.api import Satrec, jday
import numpy as np
from datetime import datetime, timezone

def fetch_tle_by_catnr(catnr):
    """
    Try to fetch TLE for a given NORAD CATNR from common public endpoints.
    Returns (line1, line2) or None on failure.
    """
    urls = [
        f"https://celestrak.com/NORAD/elements/gp.php?CATNR={catnr}&FORMAT=tle",
        f"https://celestrak.com/satcat/tle.php?CATNR={catnr}",
        f"https://celestrak.com/NORAD/elements/gp.php?CATNR={catnr}"
    ]
    for url in urls:
        try:
            resp = requests.get(url, timeout=10)
            if resp.status_code == 200 and resp.text.strip():
                lines = [ln.strip() for ln in resp.text.splitlines() if ln.strip()]
                if len(lines) >= 2:
                    l1 = None; l2 = None
                    for i in range(len(lines)-1):
                        if lines[i].startswith('1 ') and lines[i+1].startswith('2 '):
                            l1, l2 = lines[i], lines[i+1]
                    if l1 is None:
                        l1, l2 = lines[0], lines[1]
                    return l1, l2
        except Exception as e:
            pass
    return None

def fetch_tle_by_name(name):
    # attempt group fetch that often contains common satellites like 'ISS'
    urls = [
        f"https://celestrak.com/NORAD/elements/gp.php?NAME={name}&FORMAT=tle",
        f"https://celestrak.com/NORAD/elements/gp.php?GROUP=stations&FORMAT=tle"
    ]
    for url in urls:
        try:
            resp = requests.get(url, timeout=10)
            if resp.status_code == 200 and resp.text.strip():
                lines = [ln.strip() for ln in resp.text.splitlines() if ln.strip()]
                for i in range(len(lines)-1):
                    block = lines[i] + " " + (lines[i+1] if i+1 < len(lines) else '')
                    if name.upper() in block.upper() and lines[i].startswith('1 ') and lines[i+1].startswith('2 '):
                        return lines[i], lines[i+1]
                if len(lines) >= 2:
                    return lines[0], lines[1]
        except Exception as e:
            pass
    return None

def parse_tle_to_eci(tle_line1, tle_line2, when_utc=None):
    if when_utc is None:
        when_utc = datetime.now(timezone.utc)
    year = when_utc.year
    month = when_utc.month
    day = when_utc.day
    hour = when_utc.hour
    minute = when_utc.minute
    second = when_utc.second + when_utc.microsecond*1e-6
    jd, fr = jday(year, month, day, hour, minute, second)
    sat = Satrec.twoline2rv(tle_line1, tle_line2)
    e, r, v = sat.sgp4(jd, fr)
    if e != 0:
        raise RuntimeError(f"SGP4 error code {e} from sgp4.sgp4 call")
    r = np.array(r)
    v = np.array(v)
    return r, v

def eci_to_lvlh_relative(r_ref, v_ref, r_obj, v_obj):
    r_ref = np.array(r_ref); v_ref = np.array(v_ref)
    r_obj = np.array(r_obj); v_obj = np.array(v_obj)
    r_norm = np.linalg.norm(r_ref)
    if r_norm == 0:
        raise ValueError("Reference position is zero vector")
    x_hat = r_ref / r_norm
    h = np.cross(r_ref, v_ref)
    h_norm = np.linalg.norm(h)
    if h_norm == 0:
        h = np.cross(r_ref, v_ref + 1e-6)
        h_norm = np.linalg.norm(h) + 1e-12
    z_hat = h / h_norm
    y_hat = np.cross(z_hat, x_hat)
    R = np.vstack([x_hat, y_hat, z_hat])
    rel_r_eci = r_obj - r_ref
    rel_v_eci = v_obj - v_ref
    rel_r_lvlh = R.dot(rel_r_eci)
    rel_v_lvlh = R.dot(rel_v_eci)
    state = np.concatenate([rel_r_lvlh, rel_v_lvlh])
    return state

def get_two_object_lvlh(chaser_id=None, chaser_name=None, target_id=None, target_name=None, when_utc=None):
    tle_ch = None
    if chaser_id is not None:
        tle_ch = fetch_tle_by_catnr(chaser_id)
    if tle_ch is None and chaser_name is not None:
        tle_ch = fetch_tle_by_name(chaser_name)
    if tle_ch is None:
        print("Could not fetch chaser TLE. Provide TLE manually or check network.")
        return None

    tle_tgt = None
    if target_id is not None:
        tle_tgt = fetch_tle_by_catnr(target_id)
    if tle_tgt is None and target_name is not None:
        tle_tgt = fetch_tle_by_name(target_name)
    if tle_tgt is None:
        print("Could not fetch target TLE. Provide TLE manually or check network.")
        return None

    if when_utc is None:
        when_utc = datetime.now(timezone.utc)
    try:
        r_ch, v_ch = parse_tle_to_eci(tle_ch[0], tle_ch[1], when_utc=when_utc)
        r_t, v_t = parse_tle_to_eci(tle_tgt[0], tle_tgt[1], when_utc=when_utc)
    except Exception as e:
        print("Error parsing TLE to ECI position/velocity:", e)
        return None

    chaser_lvlh = eci_to_lvlh_relative(r_ch, v_ch, r_ch, v_ch)
    target_lvlh = eci_to_lvlh_relative(r_ch, v_ch, r_t, v_t)
    return {
        'when_utc': when_utc,
        'tle_chaser': tle_ch,
        'tle_target': tle_tgt,
        'r_ch_eci_km': r_ch, 'v_ch_eci_km_s': v_ch,
        'r_tgt_eci_km': r_t, 'v_tgt_eci_km_s': v_t,
        'chaser_lvlh_state': chaser_lvlh,
        'target_lvlh_state': target_lvlh
    }

# Example usage:
# Try with common objects: ISS (25544) as chaser, and a nearby catalog object as target.
res = get_two_object_lvlh(chaser_id=25544, target_id=25544)
if res is not None:
    print("Time (UTC):", res['when_utc'])
    print("Chaser TLE (first line):", res['tle_chaser'][0])
    print("Target TLE (first line):", res['tle_target'][0])
    print("Chaser LVLH state (rx,ry,rz,vx,vy,vz) [km, km/s]:\n", np.round(res['chaser_lvlh_state'], 6))
    print("Target LVLH state (target relative to chaser) [km, km/s]:\n", np.round(res['target_lvlh_state'], 6))
else:
    print("Automatic TLE ingestion failed. To proceed manually, paste two TLE lines for chaser and two for target and use parse_tle_to_eci(...).")
